### Step 1: Load data

In [3]:
import os
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from torchvision import transforms, models
from torch.utils.data import DataLoader
from ChestXRayDataset import ChestXRayDataset
from skmultilearn.model_selection import IterativeStratification
from sklearn.metrics import classification_report, precision_recall_curve
from tqdm import tqdm
import torch_optimizer as optim_mod
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
df_train = pd.read_csv('../data/data_entries/miccai2023_nih-cxr-lt_labels_train.csv')
df_val = pd.read_csv('../data/data_entries/miccai2023_nih-cxr-lt_labels_val.csv')
df_test = pd.read_csv('../data/data_entries/miccai2023_nih-cxr-lt_labels_test.csv')

In [3]:
df_train.shape, df_val.shape, df_test.shape

((78506, 22), (12533, 22), (21081, 22))

Some images exist in images folder, but not in miccai labels and opposite

To fix this problem We create list of all image IDs in both train and test _images folder
then ensure that only images withing this list are loaded to dataset

In [4]:
image_dir_train = '../data/train_images'
image_dir_test = '../data/test_images'
# Define function to get valid image ids that exist in the image directory
def get_valid_image_ids(df, image_dir):
    # Get the set of image IDs that exist in the image directory
    image_files = set(os.listdir(image_dir))  # List of all files in the image directory
    # Check if image id exists in the image directory
    valid_ids = df[df['id'].isin(image_files)]['id']
    return valid_ids

In [5]:
# Get valid image ids for train and test datasets
valid_train_ids = get_valid_image_ids(df_train, image_dir_train)
valid_val_ids = get_valid_image_ids(df_val, image_dir_train)
valid_test_ids = get_valid_image_ids(df_test, image_dir_test)

In [6]:
# Filter the DataFrames to include only the valid image IDs
df_train_valid = df_train[df_train['id'].isin(valid_train_ids)]
df_val_valid = df_val[df_val['id'].isin(valid_val_ids)]
df_test_valid = df_test[df_test['id'].isin(valid_test_ids)]

df_train_valid.shape, df_val_valid.shape, df_test_valid.shape

((78506, 22), (8018, 22), (21081, 22))

In [7]:
df_train_val = pd.concat([df_train_valid, df_val_valid], ignore_index=True)
assert df_train_val.shape[0] == df_train_valid.shape[0] + df_val_valid.shape[0]

In [8]:
df_train_val = df_train_val.copy()
#df_train_val.drop(columns=['Pneumomediastinum', 'subj_id'], inplace=True)
# Labels such Pneumomediastinum and Hernia rarely appears, but we keep them to generlize the model
df_train_val.drop(columns=['subj_id'], inplace=True)
df_train_val.shape

(86524, 21)

In [9]:
## Do same for testing
df_test_valid = df_test_valid.copy()
df_test_valid.drop(columns=['subj_id'], inplace=True)
df_test_valid.shape

(21081, 21)

### Step 2: Label encoding and create target column 

#### Step 2.1: Check for Inconsistent Rows

First, we’ll check if any rows violate the condition: if No Finding is 1, then all other categories should be 0.

In [10]:
# For train data
inconsistent_rows = df_train_val[(df_train_val['No Finding'] == 1) & (df_train_val.iloc[:, 1:-1].sum(axis=1) > 0)]

print(inconsistent_rows)

Empty DataFrame
Columns: [id, Atelectasis, Cardiomegaly, Consolidation, Edema, Effusion, Emphysema, Fibrosis, Hernia, Infiltration, Mass, Nodule, Pleural Thickening, Pneumonia, Pneumothorax, Pneumoperitoneum, Pneumomediastinum, Subcutaneous Emphysema, Tortuous Aorta, Calcification of the Aorta, No Finding]
Index: []

[0 rows x 21 columns]


#### Step 2.2: create mappings variable for all categories we have

In [11]:
# Extract all categories (exclude 'id')
categories = df_train_val.columns[1:]

# Create a mapping dictionary for categories to numbers
category_mapping = {category: idx for idx, category in enumerate(categories)}

print("Category Mapping:")
print(category_mapping)

Category Mapping:
{'Atelectasis': 0, 'Cardiomegaly': 1, 'Consolidation': 2, 'Edema': 3, 'Effusion': 4, 'Emphysema': 5, 'Fibrosis': 6, 'Hernia': 7, 'Infiltration': 8, 'Mass': 9, 'Nodule': 10, 'Pleural Thickening': 11, 'Pneumonia': 12, 'Pneumothorax': 13, 'Pneumoperitoneum': 14, 'Pneumomediastinum': 15, 'Subcutaneous Emphysema': 16, 'Tortuous Aorta': 17, 'Calcification of the Aorta': 18, 'No Finding': 19}


### Step 2.3: Create finding column

This column will contain list of all finding categories for each image, and we have the following scenarios
1. The image has no sickness ==> finding column is a list with only one item 'No Finding'
2. The image contains only one category i.g 'Hernia' ==> finding column is a list with only one item 'Hernia'
3. The image contains more than one category i.g 'Hernia' and 'Edema'...etc. ==> finding column is a list finding items 'Hernia' and 'Edema'

In [12]:
# Function to create 'finding' and 'finding_encoded' as a string based on category values
def create_finding(row):
    # Check if 'No Finding' is 1, indicating no other categories are marked
    if row['No Finding'] == 1:
        return ['No Finding'], str(category_mapping['No Finding'])  # Return encoded as a string
    
    else:
        # Generate lists of findings and their encoded values
        findings = [category for category in categories if row[category] == 1]
        encoded_findings = [str(category_mapping[category]) for category in findings]
        
        # Join encoded findings as a single string for stratification
        return findings, ','.join(encoded_findings)


In [13]:
# Apply function to create 'finding' and 'finding_encoded' columns in train_val data
df_train_val[['finding', 'finding_encoded']] = df_train_val.apply(
    lambda row: pd.Series(create_finding(row)), axis=1
)

df_train_val[['id', 'finding', 'finding_encoded']]

,id,finding,finding_encoded
0,00000001_000.png,[Cardiomegaly],1
1,00000001_001.png,"[Cardiomegaly, Emphysema]","1,5"
2,00000001_002.png,"[Cardiomegaly, Effusion]","1,4"
3,00000002_000.png,[No Finding],19
4,00000004_000.png,"[Mass, Nodule]","9,10"
...,...,...,...
86519,00030601_000.png,[Atelectasis],0
86520,00030661_000.png,[Atelectasis],0
86521,00030703_000.png,[Nodule],10
86522,00030703_001.png,[Nodule],10


In [14]:
# Apply function to create 'finding' and 'finding_encoded' columns in test data
df_test_valid[['finding', 'finding_encoded']] = df_test_valid.apply(
    lambda row: pd.Series(create_finding(row)), axis=1
)

df_test_valid[['id', 'finding', 'finding_encoded']]

,id,finding,finding_encoded
0,00000013_000.png,[No Finding],19
1,00000013_001.png,"[Emphysema, Pneumothorax, Subcutaneous Emphysema]","5,13,16"
2,00000013_002.png,"[Emphysema, Pneumothorax, Subcutaneous Emphysema]","5,13,16"
3,00000013_003.png,[Pleural Thickening],11
4,00000013_004.png,"[Effusion, Emphysema, Infiltration, Pneumothor...","4,5,8,13,16"
...,...,...,...
21076,00030800_000.png,[No Finding],19
21077,00030802_000.png,[No Finding],19
21078,00030803_000.png,[No Finding],19
21079,00030804_000.png,[No Finding],19


In [15]:
class_counts_train = df_train_val.sum(axis=0)
class_counts_train

id                            00000001_000.png00000001_001.png00000001_002.p...
Atelectasis                                                                8280
Cardiomegaly                                                               1707
Consolidation                                                              2852
Edema                                                                      1378
Effusion                                                                   8659
Emphysema                                                                  1423
Fibrosis                                                                   1251
Hernia                                                                      141
Infiltration                                                              13782
Mass                                                                       4034
Nodule                                                                     4708
Pleural Thickening                      

In [16]:
class_counts_test = df_test_valid.sum(axis=0)
class_counts_test

id                            00000013_000.png00000013_001.png00000013_002.p...
Atelectasis                                                                2700
Cardiomegaly                                                                868
Consolidation                                                              1497
Edema                                                                       751
Effusion                                                                   3735
Emphysema                                                                   917
Fibrosis                                                                    365
Hernia                                                                       62
Infiltration                                                               5159
Mass                                                                       1329
Nodule                                                                     1305
Pleural Thickening                      

### Step 3: Create subsets and DataLoaders for the training, validation, and test sets

In [17]:
# Image transformation for training and validation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [18]:
subset_ratio = 0.99 # start with 30% of data
train_val_ratio = 0.8 # For train, val ratio

#### Step 3.1 Take a Stratified 30% Subset Based on finding_encoded

**First: create the subset**

In [19]:
label_matrix = df_train_val['finding_encoded'].str.get_dummies(sep=',')

# Initialize IterativeStratification
stratifier = IterativeStratification(n_splits=2, order=1, sample_distribution_per_fold=[subset_ratio, 1 - subset_ratio]) # 1 - 0.3 = 0.7

# Perform the stratified split
train_indices, subset_indices = next(stratifier.split(df_train_val, label_matrix))

# Create the subset and remaining dataframes
subset_df_train_val = df_train_val.iloc[subset_indices].reset_index(drop=True)
remaining_df_train_val = df_train_val.iloc[train_indices].reset_index(drop=True)

In [20]:
# After the first stratified split (Subset and Remaining)
initial_size = len(df_train_val)
print(f"Initial size of df_train_val: {initial_size} rows")

subset_size = len(subset_df_train_val)
remaining_size = len(remaining_df_train_val)
print(f"Subset size for training (80%): {subset_size} rows")
print(f"Remaining size not used (20%): {remaining_size} rows")

Initial size of df_train_val: 86524 rows
Subset size for training (80%): 85643 rows
Remaining size not used (20%): 881 rows


In [21]:
assert initial_size == subset_size + remaining_size

In [22]:
subset_df_train_val.shape

(85643, 23)

**Then: split to train, val**

In [23]:
# Perform the stratified split
label_matrix_2 = subset_df_train_val['finding_encoded'].str.get_dummies(sep=',')
stratifier_2 = IterativeStratification(n_splits=2, order=1, sample_distribution_per_fold=[train_val_ratio, 1 - train_val_ratio]) 
val_indices, train_indices = next(stratifier_2.split(subset_df_train_val, label_matrix_2))

train_df = subset_df_train_val.iloc[train_indices].reset_index(drop=True)
val_df = subset_df_train_val.iloc[val_indices].reset_index(drop=True)

# Verify the sizes
print(f"Train size (80% of subset): {len(train_df)} rows")
print(f"Val size (20% of subset): {len(val_df)} rows")

Train size (80% of subset): 68442 rows
Val size (20% of subset): 17201 rows


#### Step 3.2: Create dataset objects


In [24]:
train_dataset = ChestXRayDataset(dataframe=train_df, image_dir=image_dir_train, category_mapping=category_mapping, transform=train_transform)

val_dataset = ChestXRayDataset(dataframe=val_df, image_dir=image_dir_train, category_mapping=category_mapping, transform=val_transform)

test_dataset = ChestXRayDataset(dataframe=df_test_valid, image_dir=image_dir_test, category_mapping=category_mapping, transform=val_transform)

#### Step 3.3: Create dataloaders

In [25]:
# Create Dataloaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [26]:
print(f"Training DataLoader: {len(train_loader)} batches")
print(f"Validation DataLoader: {len(val_loader)} batches")
print(f"Testing DataLoader: {len(test_loader)} batches")

Training DataLoader: 2139 batches
Validation DataLoader: 538 batches
Testing DataLoader: 659 batches


### Step 4: Define the model
We use pretrained ResNet
- We add linear layer to predict multi-label ( more than one category per image)
- ResNet-50 designed to output 2048 features
- We add two Linear layer to avoid going down from 2048 features to only 20 which may cause instability and overfitting

**Note**: Different additional params/functionality to the network i.g Dropout, normalization...etc. have been tested

In [27]:
# Number of unique categories (labels) in the 'finding_encoded' column
num_classes = len(df_train_val['finding_encoded'].str.get_dummies(sep=',').columns)
print(num_classes)

20


In [28]:
model = models.resnet50(weights='IMAGENET1K_V2') 
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),  # to reduce complexity gradually (2048, 512)
    nn.BatchNorm1d(512),
    nn.ReLU(), 
    nn.Dropout(0.4),
    nn.Linear(512, 256), 
    nn.BatchNorm1d(256),
    nn.ReLU(), 
    nn.Dropout(0.4),
    nn.Linear(256, num_classes),  # Exclude 'id', 20 different categories
    nn.Sigmoid()  # Sigmoid for multi-label classification
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model.to(device)

cuda


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Step 5: Train, Validate and test the model

[Referance about BCEWithLogitsLoss()](https://medium.com/@sahilcarterr/why-nn-bcewithlogitsloss-numerically-stable-6a04f3052967)

nn.BCEWithLogitsLoss():

1. This loss function is more efficient because it combines the sigmoid activation and binary cross-entropy loss into a single function.
2. It expects the raw logits (not passed through sigmoid) as input and applies the sigmoid internally.
3. It's numerically more stable and faster than using nn.BCELoss() with a separate sigmoid.

nn.BCELoss():

1. This loss function expects the model's output to be probabilities in the range [0, 1], so it requires you to apply a sigmoid activation to the model's output beforehand.
2. The formula for binary cross-entropy is applied after transforming the raw logits into probabilities using the sigmoid function.

***Define weighted classes***

In [29]:
categories  = train_df['finding_encoded'].str.split(',').explode().value_counts()
categories.index = categories.index.astype(int)
categories = categories.sort_index()
categories

finding_encoded
0      6558
1      1352
2      2258
3      1091
4      6858
5      1127
6       986
7       112
8     10915
9      3195
10     3729
11     1776
12      694
13     2089
14      183
15       73
16      810
17      502
18      312
19    39399
Name: count, dtype: int64

In [30]:
total = len(train_df)
total

68442

In [31]:
proportions =  categories / total
proportions

finding_encoded
0     0.095818
1     0.019754
2     0.032991
3     0.015941
4     0.100202
5     0.016466
6     0.014406
7     0.001636
8     0.159478
9     0.046682
10    0.054484
11    0.025949
12    0.010140
13    0.030522
14    0.002674
15    0.001067
16    0.011835
17    0.007335
18    0.004559
19    0.575655
Name: count, dtype: float64

In [32]:
weights = 1 / proportions
weights

finding_encoded
0      10.436414
1      50.622781
2      30.310895
3      62.733272
4       9.979878
5      60.729370
6      69.413793
7     611.089286
8       6.270454
9      21.421596
10     18.353982
11     38.537162
12     98.619597
13     32.763045
14    374.000000
15    937.561644
16     84.496296
17    136.338645
18    219.365385
19      1.737151
Name: count, dtype: float64

In [33]:
normalized_weights = weights / weights.sum()
normalized_weights

finding_encoded
0     0.003630
1     0.017609
2     0.010544
3     0.021822
4     0.003472
5     0.021125
6     0.024146
7     0.212569
8     0.002181
9     0.007452
10    0.006384
11    0.013405
12    0.034305
13    0.011397
14    0.130097
15    0.326133
16    0.029392
17    0.047426
18    0.076307
19    0.000604
Name: count, dtype: float64

In [34]:
weights_tensor = torch.tensor(weights.values, dtype=torch.float32).to(device)
weights_tensor

tensor([ 10.4364,  50.6228,  30.3109,  62.7333,   9.9799,  60.7294,  69.4138,
        611.0893,   6.2705,  21.4216,  18.3540,  38.5372,  98.6196,  32.7630,
        374.0000, 937.5616,  84.4963, 136.3386, 219.3654,   1.7372],
       device='cuda:0')

In [35]:
# Use BCEWithLogitsLoss for multi-label classification
criterion = nn.BCEWithLogitsLoss(pos_weight=weights_tensor)   # Combined Sigmoid + Binary Cross-Entropy Loss

In [36]:
#optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.8) 

In [37]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
lookahead_optimizer = optim_mod.Lookahead(optimizer, k=5, alpha=0.5)

scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.7)

In [38]:
# Initialize lists to store loss and accuracy per epoch
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

threshold = 0.5

In [39]:
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    for images, labels, _ in tqdm(dataloader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels.float())
        running_loss += loss.item()

        loss.backward()
        optimizer.step()

        all_preds.append(torch.sigmoid(outputs).detach().cpu())
        all_labels.append(labels.cpu())

    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()
    
    # Print each label, prediction, and corresponding pos_weight (for debug and ensure everything as excepted)
        #preds = torch.sigmoid(outputs).detach().cpu()
        #for i in range(len(labels)):  # Loop through each image in the batch
            #label = labels[i].cpu().numpy()
            #pred = preds[i].numpy()
            #print(f"Label: {label}, Prediction: {pred}, Pos_Weight: {weights_tensor.cpu().numpy()}")

    # Calculate F1 scores and thresholds for each class
    best_thresholds = []
    for i in range(all_labels.shape[1]):
        precision, recall, thresholds = precision_recall_curve(all_labels[:, i], all_preds[:, i])
        f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
        best_threshold = thresholds[np.argmax(f1_scores)]
        best_thresholds.append(best_threshold)

    # Apply class-specific thresholds
    thresholded_preds = np.column_stack([(all_preds[:, i] > best_thresholds[i]).astype(float) for i in range(all_labels.shape[1])])

    correct_predictions = (thresholded_preds == all_labels).all(axis=1).sum()
    avg_loss = running_loss / len(dataloader)
    avg_accuracy = (correct_predictions / all_labels.shape[0]) * 100

    report = classification_report(all_labels, thresholded_preds, zero_division=0, output_dict=True)
    precision, recall, f1 = report['macro avg']['precision'], report['macro avg']['recall'], report['macro avg']['f1-score']

    return avg_loss, avg_accuracy, precision, recall, f1, best_thresholds, correct_predictions


In [40]:
def validate(model, dataloader, criterion, device, best_thresholds):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels, _ in tqdm(dataloader, desc="Validating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels.float())
            running_loss += loss.item()

            all_preds.append(torch.sigmoid(outputs).cpu())
            all_labels.append(labels.cpu())

    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()

    # Apply class-specific thresholds
    thresholded_preds = np.column_stack([(all_preds[:, i] > best_thresholds[i]).astype(float) for i in range(all_labels.shape[1])])

    correct_predictions = (thresholded_preds == all_labels).all(axis=1).sum()
    avg_loss = running_loss / len(dataloader)
    avg_accuracy = (correct_predictions / all_labels.shape[0]) * 100

    report = classification_report(all_labels, thresholded_preds, zero_division=0, output_dict=True)
    precision, recall, f1 = report['macro avg']['precision'], report['macro avg']['recall'], report['macro avg']['f1-score']

    return avg_loss, avg_accuracy, precision, recall, f1, correct_predictions

In [41]:
def test(model, dataloader, criterion, device, label_names, best_thresholds):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels, _ in tqdm(dataloader, desc="Testing"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels.float())
            running_loss += loss.item()

            all_preds.append(torch.sigmoid(outputs).cpu())
            all_labels.append(labels.cpu())

    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()

    # Dynamic threshold for each label based on train data
    thresholded_preds = np.column_stack([(all_preds[:, i] > best_thresholds[i]).astype(float) for i in range(all_labels.shape[1])])

    print("\nFinding Best Thresholds for Each Class (Reported Only):")
    for i, threshold in enumerate(best_thresholds):
        print(f"Class '{label_names[i]}': Best Threshold = {threshold:.4f}")

    default_preds = (all_preds > 0.5).astype(float)

    print("\nClassification Report with Best Thresholds:")
    report_best = classification_report(all_labels, thresholded_preds, target_names=label_names, zero_division=0)
    print(report_best)

    print("\nClassification Report with Default Threshold (0.5):")
    report_default = classification_report(all_labels, default_preds, target_names=label_names, zero_division=0)
    print(report_default)

    avg_loss = running_loss / len(dataloader)

    return avg_loss, report_best, report_default, best_thresholds

In [42]:
num_epochs = 100

# Early stopping parameters
patience = 7 # wait 
best_val = float("inf")
epochs_no_improve = 0

# for plotting
num_epochs_runned = 0

In [43]:
for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    
    # Train the model for one epoch
    train_loss, train_accuracy, train_precision, train_recall, train_f1, train_best_threshold, train_correct_predictions = train(model, train_loader, criterion, optimizer, device)
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
      f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, "
      f"Train F1: {train_f1:.4f}, Train Best Threshold: {train_best_threshold:.4f}, "
      f"Train correct predictions: {train_correct_predictions}")



    # Validate the model after training
    val_loss, val_accuracy, val_precision, val_recall, val_f1, val_correct_predictions = validate(model, val_loader, criterion, device, train_best_threshold)

    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, "
      f"Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, "
      f"Val F1: {val_f1:.4f}, Val correct predictions: {val_correct_predictions}")


    # Step the Lookahead optimizer
    lookahead_optimizer.step()
    # Step the ReduceLROnPlateau scheduler
    scheduler.step(val_loss)

    print(f"Epoch {epoch + 1}: Learning rate = {scheduler.get_last_lr()[0]}")

    num_epochs_runned += 1

    # Validation loss has improved
    if val_loss < best_val:
        best_val = val_loss
        epochs_no_improve = 0  # Reset the counter if we see improvement
        
        # Save the model checkpoint
        torch.save(model.state_dict(), "singular_weighted_classes.pth")
        print("Model improved and saved.")
    else:
        print("Model did not improved")
        epochs_no_improve += 1  # Increment counter if no improvement

    # Early stopping
    if epochs_no_improve == patience:
        print("Early stopping triggered")
        break

    print("-" * 150)

Epoch [1/100]


Training: 100%|██████████| 2139/2139 [05:16<00:00,  6.77it/s]


Train Loss: 1.3262, Train Accuracy: 44.1352, Train Precision: 0.0758, Train Recall: 0.0912, Train F1: 0.0683, Train Best Threshold: 0.6660, Train correct predictions: 30207


Validating: 100%|██████████| 538/538 [01:23<00:00,  6.45it/s]


Total prediction: 17201  Correct preds: 7530  all labels size: (17201, 20)
Val Loss: 1.3208, Val Accuracy: 43.7765, Val Precision: 0.0789, Val Recall: 0.1115, Val F1: 0.0691, Val correct predictions: 7530
Epoch 1: Learning rate = 0.001
Model improved and saved.
------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch [2/100]


...

Total prediction: 17201  Correct preds: 5439  all labels size: (17201, 20)
Val Loss: 1.2772, Val Accuracy: 31.6203, Val Precision: 0.1114, Val Recall: 0.2285, Val F1: 0.0980, Val correct predictions: 5439
Epoch 41: Learning rate = 0.00049
Model did not improved
------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch [42/100]


Training: 100%|██████████| 2139/2139 [05:17<00:00,  6.73it/s]


Train Loss: 1.2710, Train Accuracy: 33.7746, Train Precision: 0.1114, Train Recall: 0.2119, Train F1: 0.0947, Train Best Threshold: 0.7244, Train correct predictions: 23116


Validating: 100%|██████████| 538/538 [01:23<00:00,  6.41it/s]


Total prediction: 17201  Correct preds: 5043  all labels size: (17201, 20)
Val Loss: 1.2781, Val Accuracy: 29.3181, Val Precision: 0.1692, Val Recall: 0.2146, Val F1: 0.0798, Val correct predictions: 5043
Epoch 42: Learning rate = 0.00049
Model did not improved
Early stopping triggered


In [44]:
# Define label names for better readability in the report
label_names = [
    'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 
    'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 
    'Pleural Thickening', 'Pneumonia', 'Pneumothorax', 'Pneumoperitoneum', 
    'Pneumomediastinum', 'Subcutaneous Emphysema', 'Tortuous Aorta', 
    'Calcification of the Aorta', 'No Finding'
]

In [45]:
# Now, evaluate the model on the test set
model.load_state_dict(torch.load("singular_weighted_classes.pth", weights_only=True))

test_loss, test_report_best, test_report_default, test_best_thresholds = test(model, test_loader, criterion, device, label_names, train_best_threshold)

print(f"Test Loss: {test_loss:.4f}")

Testing: 100%|██████████| 659/659 [01:37<00:00,  6.77it/s]



Finding Best Thresholds for Each Class:
Class 'Atelectasis': Best Threshold = 0.5944, Max F1-Score = 0.2605
Class 'Cardiomegaly': Best Threshold = 0.6693, Max F1-Score = 0.1153
Class 'Consolidation': Best Threshold = 0.7205, Max F1-Score = 0.1982
Class 'Edema': Best Threshold = 0.7311, Max F1-Score = 0.1351
Class 'Effusion': Best Threshold = 0.6934, Max F1-Score = 0.3803
Class 'Emphysema': Best Threshold = 0.6550, Max F1-Score = 0.0975
Class 'Fibrosis': Best Threshold = 0.6631, Max F1-Score = 0.0837
Class 'Hernia': Best Threshold = 0.7311, Max F1-Score = 0.0519
Class 'Infiltration': Best Threshold = 0.6226, Max F1-Score = 0.4316
Class 'Mass': Best Threshold = 0.6338, Max F1-Score = 0.1384
Class 'Nodule': Best Threshold = 0.5079, Max F1-Score = 0.1457
Class 'Pleural Thickening': Best Threshold = 0.6391, Max F1-Score = 0.1145
Class 'Pneumonia': Best Threshold = 0.7266, Max F1-Score = 0.0737
Class 'Pneumothorax': Best Threshold = 0.5538, Max F1-Score = 0.2274
Class 'Pneumoperitoneum': Be

In [4]:
plt.plot(range(num_epochs_runned), train_losses, label="Train Loss")
plt.plot(range(num_epochs_runned), val_losses, label="Validation Loss")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss singular weighted classes')
plt.show()

# Plot training and validation accuracy
plt.plot(range(num_epochs_runned), train_accuracies, label="Train Accuracy")
plt.plot(range(num_epochs_runned), val_accuracies, label="Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

# Save the results in a DataFrame for reporting
results_df = pd.DataFrame({
    'Epoch': range(1, num_epochs_runned + 1),
    'Train Loss': train_losses,
    'Train Accuracy': train_accuracies,
    'Val Loss': val_losses,
    'Val Accuracy': val_accuracies
})

print(results_df)

NameError: name 'num_epochs_runned' is not defined

<Figure size 600x400 with 0 Axes>